In [1]:
#Installing accelrate
!pip install accelerate

In [2]:
#importing required packages for LLM
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
import re,json

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
#Loading the Tokenizer and LLM Model Google Flan T5 -XL
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl", device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
#Output Processing - Removing Tags
clean = re.compile('<.*?>')

In [5]:
def analyse(review):
    result={}
    #result["Review"] = review
    #As running in Kaggle GPU with Flan T5 XL, Tried identifying each items in individual prompt. If we use bigger modles, all these can be done in single prompt
    prompt1="""Please analyze and identify the sentiment as postive, negative, suggestions the following user review:
    Review: """ + review
    prompt2="""Provide two important businsess action items to do based on below review to improve business:
    Review: """ +review
    prompt3="""Based on below review, what is the great feature and good thing"
    Review: """ +review
    prompt4= """Please analyze and provide which segment we can classify this review like camera issue, display problem, Best camera, best display,battery issues etc:
    Review: """ +review
    input_ids1 = tokenizer(prompt1, return_tensors="pt").input_ids.to("cuda")
    outputs1 = model.generate(input_ids1)
    result["Sentiment"] = re.sub(clean,'',tokenizer.decode(outputs1[0])).strip().upper()
    if result["Sentiment"]=="NEGATIVE":
        input_ids2 = tokenizer(prompt2, return_tensors="pt").input_ids.to("cuda")
        outputs2 = model.generate(input_ids2)
        result["ActionItems"] = re.sub(clean,'',tokenizer.decode(outputs2[0])).strip()
    else:
        input_ids3 = tokenizer(prompt3, return_tensors="pt").input_ids.to("cuda")
        outputs4 = model.generate(input_ids3)
        result["PostiveFeatures"] = re.sub(clean,'',tokenizer.decode(outputs4[0])).strip()
    input_ids4 = tokenizer(prompt4, return_tensors="pt").input_ids.to("cuda")
    outputs4 = model.generate(input_ids4)
    result["category"] = re.sub(clean,'',tokenizer.decode(outputs4[0])).strip()
    return result

In [6]:
#Negative English Feedback - 
feedback="I am very disappointed with this phone, I usually don't have issues with this brand but this phone's battery won't stand for 5 hrs. even with simple use, the battery drains so fast."
print(json.dumps(analyse(feedback), indent=4))

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{
    "Sentiment": "NEGATIVE",
    "ActionItems": "Review the phone's battery life",
    "category": "battery issues"
}


In [7]:
#Negative French Feedback
feedback="Je suis tres déçu de ce telephone, je n'ai generalement pas de problemes avec cette marque mais la batterie de ce telephone ne tient pas 5 heures. meme avec une utilisation simple, la batterie se decharge tres rapidement."
print(json.dumps(analyse(feedback), indent=4))

{
    "Sentiment": "NEGATIVE",
    "ActionItems": "Review the phone's battery life",
    "category": "battery issues"
}


In [8]:
#Postive English Feedback
feedback="i like the phone a lot, it has lot of good features. It has the great camera"
print(json.dumps(analyse(feedback), indent=4))

{
    "Sentiment": "POSITIVE",
    "PostiveFeatures": "it has the great camera",
    "category": "Best camera"
}


In [9]:
#Positive French Feedback
feedback="j aime beaucoup le telephone, il a beaucoup de bonnes fonctionnalites. Il a le super appareil photo"
print(json.dumps(analyse(feedback), indent=4))

{
    "Sentiment": "POSITIVE",
    "PostiveFeatures": "j aime beaucoup le telephone, il a beaucoup de bonnes fonctionnal",
    "category": "Best camera"
}


In [10]:
!pip install pyngrok 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19885 sha256=41b57400eec7f6a81c43aa9cbea41d2c37e088d2b5cec39acd5dd9e3df1b7890
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [11]:
!pip install dash jupyter_dash dash_core_components dash_bootstrap_components dash_html_components pyngrok 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.6/220.6 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 20.1 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 2.3.6
    Uninstalling Werkzeug-2.3.6:
      Successfully uninstalled Werkzeug-2.3.6
  Attempting uninstall: Flask
    Found existing installation: Flask 2.3.2
    Uninstalling Flask-2.3.2:
      Successfully uninstalled Flask-2.3.2


In [12]:
!ngrok help

NAME:
  ngrok - tunnel local ports to public URLs and inspect traffic

USAGE:
  ngrok [command] [flags]

DESCRIPTION: 
  ngrok exposes local networked services behinds NATs and firewalls to the
  public internet over a secure tunnel. Share local websites, build/test
  webhook consumers and self-host personal services.
  Detailed help for each command is available with 'ngrok help <command>'.
  Open http://localhost:4040 for ngrok's web interface to inspect traffic.

Author:
  ngrok - <support@ngrok.com>

TERMS OF SERVICE: https://ngrok.com/tos

EXAMPLES: 
  ngrok http 80                           # secure public URL for port 80 web server
  ngrok http --domain baz.ngrok.dev 8080  # port 8080 available at baz.ngrok.dev
  ngrok http foo.dev:80                   # tunnel to host:port instead of localhost
  ngrok http https://localhost            # expose a local https server
  ngrok tcp 22                            # tunnel arbitrary TCP traffic to port 22
  ngrok tls --domain=foo.com 44

In [13]:
!ngrok authtoken 2UezvPrva7mwe0LYzeaVD1Kr1kj_2gEr4wVBqcJWZVeaky48v

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [14]:
from pyngrok import ngrok
tunnel = ngrok.connect(8050)

Exception in thread Thread-7 (_monitor_process):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/site-packages/pyngrok/process.py", line 146, in _monitor_process
    self._log_line(self.proc.stdout.readline())
  File "/opt/conda/lib/python3.10/encodings/ascii.py", line 26, in decode
    return codecs.ascii_decode(input, self.errors)[0]
UnicodeDecodeError: 'ascii' codec can't decode byte 0xc2 in position 184: ordinal not in range(128)


In [15]:
tunnel

<NgrokTunnel: "https://31e4-130-211-237-52.ngrok-free.app" -> "http://localhost:8050">

In [16]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from jupyter_dash import JupyterDash
from dash.dependencies import Input,Output,State
from dash.exceptions import PreventUpdate
import requests,json
from dash import dash_table
import pandas as pd
import queue
from threading import Thread
import base64
import re

/tmp/ipykernel_24/1061526865.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/tmp/ipykernel_24/1061526865.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [17]:
app=JupyterDash(__name__,server_url=tunnel.public_url)

/opt/conda/lib/python3.10/site-packages/dash/dash.py:525: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



In [18]:
image_filename="/kaggle/input/header/header.png"
with open(image_filename,"rb") as image_file:
    img_data=base64.b64encode(image_file.read())
    img_data=img_data.decode()
    img_data="{}{}".format("data:image/png;base64, ",img_data)

In [19]:
app.title="GENAI Feedback Service"
app.layout = html.Div([
    html.Img(id="tag_id",src=img_data,style={'textAlign':'left','width':'100%','height':'10%'}),
    html.Br(),
    html.Br(),
    html.P("Please enter your feedback",style={'verticalAlign':'middle','textAlign':'centre','offset':3}),
    dbc.Row([
        dbc.Col(dbc.Textarea(id='search',size=60,placeholder="Please enter your feedback",style={"height":50}),width={'size':6,'offset':3},style={'font-weight':'bold'}),
    ]),
    html.Br(),
    html.Button(id='Set-val',n_clicks=0,children="Submit"),
    html.Br(),
    dbc.Spinner(html.Div(id="answers"),),
],style={
   'verticalAlign':'middle',
    'textAlign':'centre',
    'background-color':'#FFFFFF',
    'position':'fixed',
    'width':'100%',
    'height':'100%',
    'top':'0px',
    'left':'0px',
    'z-index':"1000",
    'background-size':'cover',
    'overflow':'scroll'
})

In [20]:
@app.callback(Output("answers","chidren"),
             [Input('Set-val','n_clicks')],
             State("Search","value"),)

def filter_flag(click_count,search=""):
    try:
        if str(search).strip()!=None:
            r1=analyse(search)
            print(r1)
            card_data=[]
            card_data.append(html.P("Analysis of the User Feedback"))
            card_data.append(html.P(" "))
            card_data.append(html.P("Feedback: "+search))
            card_data.append(html.P(" "))
            card_data.append(html.P("Sentiment: " +r1["Sentiment"]))
            card_data.append(html.P(" "))
            if r1[0]=="NEGATIVE":
                card_data.append(html.P("Action Items: ",r1["ActionItems"]))
            else:
                card_data.append(html.P("PostiveFeatures: ",r1['PostiveFeatures']))
            card_data.append(html.P(" "))
            card_data.append(html.P("Category: " +r1['category']))
            card_data.append(html.P(" "))
            card1=dbc.Card(
            [
                dbc.CardHeader("GenAI Feedback Analysis"),
                dbc.cardBody(card_data),
            ],style={'textAlign':'left'},)
            
            lay=html.Dic(
            [
                html.Hr(),
                dbc.Row([dbc.Col(html.Div(card1),width={'offset':3,'size':6,'textAlign':'left'},)]),
                dbc.Row([dbc.Col(html.Dic([html.Br()]),)]),
                html.Hr()
            ])
            return lay
        else:
            return ""
    except Exception as e:
        print(e)
        return str(e)

In [21]:
app.run_server(mode='external')

Dash app running on https://31e4-130-211-237-52.ngrok-free.app/
